In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [109]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 200
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
embed_size = 32
# ------------

In [64]:
device


'cpu'

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-14 18:57:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2026-01-14 18:57:40 (9.41 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [89]:
256*eval_iters

51200

In [8]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [43]:
class modeldata:
    def __init__(self, data, tag):
        self.data =data
        self.tag = tag
    


In [44]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = modeldata(data[:n], "train")
val_data = modeldata(data[n:], "val")

In [71]:
class dataloaderlite:
    def __init__(self, data, block_size, batch_size, shuffle=True, tag=None):
        self.data = data
        self.L = block_size
        self.B= batch_size
        self.current_pos = 0
        self.shuffle = shuffle
        self.N = len(data)
        self.tag = tag

    def __iter__(self):
        return self

    def __next__(self):
        if self.shuffle:
            # shuffle mode: sample random batches
            idx = torch.randint(0,self.N - self.L - 1, (self.B,))
        else:
            # sequential mode: get batches in order, mainly for eval
            if self.current_pos + self.B * self.L >= self.N:
                self.current_pos = 0  # reset pointer if we reach the end
                raise StopIteration
            idx = torch.arange(self.current_pos, self.current_pos + self.B * self.L, self.L)
            self.current_pos += self.B * self.L
        x = torch.stack([self.data[i:i+self.L] for i in idx])
        y = torch.stack([self.data[i+1:i+self.L+1] for i in idx])    
        return x.to(device), y.to(device)

0

In [110]:
train_loader = dataloaderlite(train_data.data, block_size, batch_size, shuffle=True, tag="train")
val_loader = dataloaderlite(val_data.data, block_size, batch_size, shuffle=False, tag="val")

In [94]:
@torch.no_grad()
def loss_estimation(model):
    output = {}
    model.eval()
    train_data_loader_estimation = dataloaderlite(train_data.data, block_size, batch_size, shuffle=True, tag="train")
    val_data_loader_estimation = dataloaderlite(val_data.data, block_size, batch_size, shuffle=True, tag="val")
    for data_obj in [train_data_loader_estimation, val_data_loader_estimation]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = data_obj.__next__()
            logits, loss = model(x, y)
            losses[k] = loss.item()
        output[data_obj.tag] = losses.mean().item()
    model.train()
    return output

    

In [116]:
class BaselineModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super().__init__()
        self.embed_table = nn.Embedding(vocab_size, embed_size)
        self.FFN = nn.Linear(embed_size, vocab_size)
    
    def forward(self, id_matrix, targets=None):
        # id_matrix: (B, L) is the input token indices for B batches, each batch has sequence length L
        # d: the embed size
        # v: the vocab size
        B, L = id_matrix.shape
        token_emb = self.embed_table(id_matrix) # (B, L, d)
        logits = self.FFN(token_emb) # (B, L, vocab_size)
        if targets is None:
            loss = None
        else:
            B, L, v = logits.shape
            # reshape the logits and targets to compute the cross-entropy loss
            logits = logits.view(B*L, v)
            targets = targets.view(B*L)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, id_matrix, max_new_tokens):
        B, L = id_matrix.shape
        for _ in range(max_new_tokens):
            logits, _ = self(id_matrix)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, vocab_size)
            probs = F.softmax(logits, dim=-1) # (B, vocab_size)
            # sample from the distribution
            next_id = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            id_matrix = torch.cat((id_matrix, next_id), dim=1) # (B, L+1)
        return id_matrix
       

In [117]:
model = BaselineModel(vocab_size, embed_size)
model = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [118]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = loss_estimation(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = train_loader.__next__()

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3202, val loss 4.3271
step 200: train loss 3.0459, val loss 3.0642
step 400: train loss 2.7385, val loss 2.7522
step 600: train loss 2.6413, val loss 2.6614
step 800: train loss 2.5904, val loss 2.5998
step 1000: train loss 2.5581, val loss 2.5822
step 1200: train loss 2.5398, val loss 2.5654
step 1400: train loss 2.5326, val loss 2.5301
step 1600: train loss 2.5270, val loss 2.5363
step 1800: train loss 2.5097, val loss 2.5099
step 2000: train loss 2.4981, val loss 2.5178
step 2200: train loss 2.5030, val loss 2.5101
step 2400: train loss 2.4941, val loss 2.5016
step 2600: train loss 2.4840, val loss 2.5021
step 2800: train loss 2.4866, val loss 2.5107


In [119]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

In [123]:
generated_context_ids = model.generate(context, max_new_tokens=1000)


In [125]:
print(decode(generated_context_ids[0].tolist()))


ArtO:

Whir h f hatou rind
Wog ous, arrea prvetho s whenal wou hayenor fd t be He wr de, ngou herudstho can?
'd
IOOMELNG m f
Thindilthenor
FICOLI d hepe nf whes:
Gk anneeinthat ingoe t ren th'st dere shy sir ataghe
entwin, thitito UESMgis st ched Fsa ly r 'trdsith
t:
May u od bu mmprtheicess Iis.
RSTar m,
3e; y Cit, thik, mathathansuchet, and shamer:
I AOOenougCHERI d l ur, n,
Rine he amou asuth thy fd veasoun ven isil homyre yot.
WAncousines, oove piam;
I ssis y ulet arokerupifl thengfs norupt nd VTHodeeningreame tout m Caing, hi-den y tank 't akis!
DUESUCANur-irssatoumou! oul cf fit, I MEOfod ainont o, llld d or hal Iferey h fs, ave f ns I ho,
S:
The sthave earcth
DWan.
S:
d fAnourkie d ornd ar,
PAnd pabud ando'dinth Idors veate avevef verfenseso yitheed che woustof-EThalfo wsth se INILO:
T:
then I shaletig-d,

Alld bes I th id'd
Lous r'l fo I ast ave l'dyou wis ll Cinthid ll tyore s
S: anneshomy mat y wowicthin, mecen'd y t I mth itrerelere cord'neallpy med tthe tine ffomprt ru cke